In [ ]:
#setup dependencies

In [ ]:
#import csvs

In [ ]:
#set up dataframes and check output
#UGB pop and growth
#top 100 pop and growth

In [ ]:
#merge the top100 Pop and growth, output

In [ ]:
#merge UGB pop and growth, output

In [ ]:
#average populations and growths
#compare to total US population growth = .62%, https://www.multpl.com/us-population-growth-rate/table/by-year


In [ ]:
#